In [ ]:
# Import all required packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import nibabel as nib

In [ ]:
# Paths (edit these if needed)
train_csv = "/mnt/Data/hackathon/final_train.csv"
val_csv = "/mnt/Data/hackathon/final_valid.csv"
test_csv = "/mnt/Data/hackathon/final_test.csv"

# --- Load ---
train_df = pd.read_csv(train_csv)
val_df = pd.read_csv(val_csv)
test_df = pd.read_csv(test_csv)

print("Train shape:", train_df.shape)
print("Val shape:", val_df.shape)
print("Test shape:", test_df.shape)
print("\nColumns:", train_df.columns.tolist())

# --- Basic info ---
print("\nSample rows:")
display(train_df.head())

# --- Number of sweeps per study ---
sweep_cols = [c for c in train_df.columns if c.startswith('path_nifti')]
train_df['n_sweeps'] = train_df[sweep_cols].notna().sum(axis=1)
val_df['n_sweeps'] = val_df[sweep_cols].notna().sum(axis=1)
test_df['n_sweeps'] = test_df[sweep_cols].notna().sum(axis=1)

print("\nAverage number of sweeps per study:")
print("Train:", train_df['n_sweeps'].mean())
print("Val:", val_df['n_sweeps'].mean())
print("Test:", test_df['n_sweeps'].mean())

# # --- Gestational Age (GA) Distribution ---
# plt.figure(figsize=(8,5))
# plt.hist(train_df['ga'], bins=30, alpha=0.7, label='Train')
# plt.hist(val_df['ga'], bins=30, alpha=0.7, label='Validation')
# plt.xlabel('Gestational Age (weeks)')
# plt.ylabel('Count')
# plt.title('GA Distribution (Train vs Validation)')
# plt.legend()
# plt.show()

# # --- Sweep Count Distribution ---
# plt.figure(figsize=(8,5))
# plt.hist(train_df['n_sweeps'], bins=np.arange(1, 10)-0.5, alpha=0.7, label='Train')
# plt.hist(val_df['n_sweeps'], bins=np.arange(1, 10)-0.5, alpha=0.7, label='Validation')
# plt.xticks(range(1, 9))
# plt.xlabel('Number of Sweeps per Study')
# plt.ylabel('Count')
# plt.title('Distribution of Sweeps per Study')
# plt.legend()
# plt.show()

# --- Missing Data Check ---
print("\nMissing value summary:")
print(train_df.isna().sum())

# --- Sweep file existence check ---
print("\nChecking for missing NIfTI files...")
missing_paths = []
for col in sweep_cols:
    missing = train_df[col].dropna().apply(lambda p: not os.path.exists(p))
    if missing.any():
        missing_paths.extend(train_df.loc[missing, col].tolist())

print(f"Missing sweep files: {len(missing_paths)}")
if len(missing_paths) > 0:
    print("Example missing paths:", missing_paths[:5])


In [ ]:
train_df = pd.read_csv(train_csv)
val_df = pd.read_csv(val_csv)
test_df = pd.read_csv(test_csv)

# Print basic sizes
print("Sizes -> train: {}, val: {}".format(len(train_df), len(val_df)))

# Plot GA histograms (one figure per split)
for df, name in [(train_df, "Train"), (val_df, "Validation")]:
    ga = df['ga'].dropna().astype(float)
    plt.figure()
    plt.hist(ga, bins=40)
    plt.title(f"{name} GA distribution (n={len(ga)})")
    plt.xlabel("Gestational Age (days)")
    plt.ylabel("Count")
    plt.show()

# Also print simple GA stats
for df, name in [(train_df, "Train"), (val_df, "Validation")]:
    ga = df['ga'].dropna().astype(float)
    print(f"{name} GA: mean={ga.mean():.2f}, median={ga.median():.2f}, std={ga.std():.2f}, min={ga.min():.1f}, max={ga.max():.1f}")

In [ ]:
def analyze_sweep_frames(csv_path, dataset_name="Dataset"):
    df = pd.read_csv(csv_path)
    sweep_cols = [c for c in df.columns if c.startswith('path_nifti')]
    
    # Pick a random study
    random_row = df.sample(1).iloc[0]
    study_id = random_row['study_id']
    sweep_paths = random_row[sweep_cols].dropna().tolist()
    
    frame_counts = []
    for path in sweep_paths:
        try:
            img = nib.load(path).get_fdata()
            num_frames = img.shape[-1]
            frame_counts.append(num_frames)
        except Exception as e:
            print(f"⚠️ Could not read {path}: {e}")
    
    if len(frame_counts) == 0:
        print(f"No valid sweeps found for {study_id} in {dataset_name}")
        return
    
    avg_frames = np.mean(frame_counts)
    print(f"\n📊 {dataset_name}: {study_id}")
    print(f"Number of sweeps: {len(frame_counts)}")
    print(f"Frames per sweep: {frame_counts}")
    print(f"Average frames per sweep: {avg_frames:.2f}")
    
    # Optional visualization
    plt.figure()
    plt.bar(range(len(frame_counts)), frame_counts)
    plt.title(f"{dataset_name}: Frames per Sweep for {study_id}")
    plt.xlabel("Sweep index")
    plt.ylabel("Number of frames")
    plt.show()

# Run for each dataset
analyze_sweep_frames(train_csv, "Training Set")
analyze_sweep_frames(val_csv, "Validation Set")
analyze_sweep_frames(test_csv, "Testing Set")


In [ ]:
random.seed(42)
np.random.seed(42)

def get_sweep_columns(df):
    return [c for c in df.columns if c.startswith("path_nifti")]

def load_nifti_safe(path):
    """Loads a NIfTI file and returns array + error string if any."""
    try:
        img = nib.load(path)
        arr = img.get_fdata().astype(np.float32)
        if arr.size == 0:
            return None, "empty"
        if np.isnan(arr).any():
            return None, "contains_nan"
        return arr, None
    except Exception as e:
        return None, f"load_error:{repr(e)}"


In [ ]:
# Pick random Patient
sampled_patients = {
    "Train": train_df.sample(1, random_state=42),
    "Validation": val_df.sample(1, random_state=42),
    "Test": test_df.sample(1, random_state=42)
}

for name, df in sampled_patients.items():
    sid = df.iloc[0].get('study_id', f"{name}_sample")
    print(f"{name} random patient selected: {sid}")


In [ ]:
REP_FRAME_FN = lambda arr: arr[..., arr.shape[-1]//2]  # pick middle frame

def plot_intensity_hist_for_patient(df, split_name):
    sweep_cols = get_sweep_columns(df)
    row = df.iloc[0]
    sid = row.get('study_id', f"{split_name}_sample")
    paths = [row[c] for c in sweep_cols if pd.notna(row[c]) and str(row[c]).strip() != ""]
    print(f"\n--- {split_name}: {sid} ---")
    print(f"Total sweeps: {len(paths)}")

    for p in paths:
        arr, err = load_nifti_safe(p)
        if err:
            print(f" {p}: {err}")
            continue

        frame = REP_FRAME_FN(arr)
        vals = frame[np.isfinite(frame)].flatten()

        if vals.size == 0:
            print(f"{p}: frame empty or non-finite values")
            continue

        print(f"🧩 {os.path.basename(p)} | mean={vals.mean():.3f}, std={vals.std():.3f}, min={vals.min():.3f}, max={vals.max():.3f}")
        plt.figure()
        plt.hist(vals, bins=200)
        plt.title(f"{split_name}: {os.path.basename(p)} intensity histogram")
        plt.xlabel("Pixel intensity")
        plt.ylabel("Count")
        plt.show()

# Run for train, val, test
for name, df in sampled_patients.items():
    plot_intensity_hist_for_patient(df, name)


In [ ]:
# Check for corrupt sweeps

def check_corrupt_sweeps(df, split_name):
    sweep_cols = get_sweep_columns(df)
    row = df.iloc[0]
    sid = row.get('study_id', f"{split_name}_sample")
    paths = [row[c] for c in sweep_cols if pd.notna(row[c]) and str(row[c]).strip() != ""]
    
    corrupt_list = []
    for p in paths:
        arr, err = load_nifti_safe(p)
        if err:
            corrupt_list.append((p, err))
    if len(corrupt_list) == 0:
        print(f"{split_name} {sid}: All sweeps passed integrity check")
    else:
        print(f"{split_name} {sid}: Found {len(corrupt_list)} corrupt/invalid sweeps")
        for p, e in corrupt_list:
            print(f"   - {p} -> {e}")

for name, df in sampled_patients.items():
    check_corrupt_sweeps(df, name)


✅ Train KA-AD-87-1: All sweeps passed integrity check
✅ Validation PN-AD-63-1: All sweeps passed integrity check
✅ Test PN-AD-66-1: All sweeps passed integrity check
